# Learning with a single neuron using Flux.jl

In this notebook, we'll use `Flux` to create a single neuron and teach it to learn, as we did by hand in notebook 10!

### Read in data and process it

Let's start by reading in our data

In [1]:
using CSV
using TextParse
using DataFrames

applecols, applecolnames = TextParse.csvread("data/Apple_Golden_1.dat", '\t')
bananacols, bananacolnames = TextParse.csvread("data/bananas.dat", '\t')
apples = DataFrame(Dict(strip(name)=>col for (name, col) in zip(applecolnames, applecols)))
bananas = DataFrame(Dict(strip(name)=>col for (name, col) in zip(bananacolnames, bananacols)));

and processing it to extract information about the red and green coloring in our images:

In [2]:
col1 = :red
col2 = :green

x_apples  = [ [apples[i, col1], apples[i, col2]] for i in 1:size(apples)[1] ]
x_bananas = [ [bananas[i, col1], bananas[i, col2]] for i in 1:size(bananas)[1] ]

xs = vcat(x_apples, x_bananas)

ys = vcat( zeros(size(x_apples)[1]), ones(size(x_bananas)[1]) );

The input data is in `xs` and the labels (true classifications as bananas or apples) in `ys`.

### Using `Flux.jl`

Now we can load `Flux` to really get going!

In [3]:
using Flux

We saw in the last notebook that σ is a built-in function in `Flux`.

Another function that is used a lot in neural networks is called `ReLU`; in Julia, the function is called `relu`.

**Exercise**: 

Use the docs to discover what `ReLU` is all about.

`relu.([-3, 3])` returns

A) [-3, 3] <br>
B) [0, 3] <br>
C) [0, 0] <br>
D) [3, 3] <br>

### Making a single neuron in Flux

Let's use `Flux` to build our 2 input and 1 output neuron:

 <img src="data/single-neuron.png" alt="Drawing" style="width: 500px;"/>

We previously put the two weights in a vector, $\mathbf{w}$. Flux instead puts weights in a $1 \times 2$ matrix. 

Previously we computed the dot product of $\mathbf{w}$ and $\mathbf{x}$ we had to use either the `dot` function or we had to transpose vector $\mathbf{w}$:

```julia
# transpose w
b = w'x
# or use dot!
b = dot(w, x)
```
If the weights are instead stored in a $1 \times 2$ matrix, `W`, we can simply multiply `W` and `x` together!

We start off with random values for our parameters and data:

In [8]:
W = rand(1, 2)

1×2 Array{Float64,2}:
 0.745903  0.987957

In [9]:
x = rand(2)

2-element Array{Float64,1}:
 0.273046
 0.433446

Note that the product of `W` and `x` will now be an array rather than a single number:

In [10]:
W * x

1-element Array{Float64,1}:
 0.631892

This means our bias `b` is an array when we're using Flux:

In [11]:
b = rand(1)

1-element Array{Float64,1}:
 0.677647

**Exercise**: 

Write a function `mypredict` that will take a single input, array `x` and use `W`, `b`, and built-in `σ` to generate an output prediction (stored as an array). This function defines our neural network!

Hint: This function will look very similar to $f_{\mathbf{w},\mathbf{b}}$ from the last notebook but has changed since our data structures to store our parameters have changed!

#### Exercise

Define a loss function called `loss`, which is what we have been calling the cost function. 

`loss` should take two inputs: a vector storing data, `x`, and a vector storing the correct "labels" for that data. `loss` should return the sum of the squares of differences between the predictions and the correct labels.

## Calculating gradients using Flux: backpropagation

For learning, we know that what we need is a way to calculate derivatives of the `loss` function with respect to the parameters `W` and `b`. So far, we have been doing that using finite differences. 

`Flux.jl` instead implements a numerical method called **backpropagation** that calculates gradients (essentially) exactly, in an automatic way.
To do so, it provides a new type of object called "tracked" arrays. These are arrays that store not only their current value, but also information about gradients, which is used by the backpropagation method.

[If you want to understand the maths behind backpropagation, we recommend e.g. [this lecture](https://www.youtube.com/watch?v=i94OvYb6noo).]

To do so, `Flux` provides a function `param` to define such objects that will contain the information for a *param*eter. 

Let's start, as usual, by setting up some random initial values for the parameters:

In [12]:
W_data = rand(1, 2)  
b_data = rand(1)

W_data, b_data

([0.522294 0.0113855], [0.0667265])

We now set up `Flux.jl` objects that will contain these values *and* their derivatives, and allow to propagate
this information around:

In [148]:
W = param(W_data)
b = param(b_data)

Tracked 1-element Array{Float64,1}:
 0.0667265

Here, `param` is a function that `Flux` provides to create an object that represents a parameter of a machine learning model.

**Exercise**: 

What type does `W` have?

A) Array (1D) <br>
B) Array (2D) <br>
C) TrackedArray (1D) <br>
D) TrackedArray (2D) <br>
E) Parameter (1D) <br>
F) Parameter (2D) <br>

#### Exercise:

`W` stores not only its current value, but also it has space to store gradient information. You can access the values and gradient of the weights by using the commands

```julia
W.data
W.grad
```

At this point, are the values of the weights or the gradient of the weights larger?

A) the values of the weights <br>
B) the gradient of the weights

**Exercise**: 

For data `x` and `y` where

```julia
x, y = [0.413759, 0.692204], [0.845677]
```
apply the loss function to `x` and `y` to give a new variable `l`. What is the type of `l`? (How many dimensions does it have?)

A) Array (0D) <br>
B) Array (1D) <br>
C) TrackedArray (0D) <br>
D) TrackedArray (1D)<br> 
E) Float64<br>
F) Int64<br>

Having set up the structure, we can now **propagate information about derivatives backwards ** from the `loss` function to all of the objects that are used to calculate it:

In [14]:
using Flux.Tracker

back!(l)   # backpropagate derivatives of the loss function

LoadError: [91mUndefVarError: l not defined[39m

and now we can look at the derivatives again:

In [15]:
W.grad

1×2 Array{Float64,2}:
 0.0  0.0

In [16]:
b.grad

1-element Array{Float64,1}:
 0.0

What are these results? They are the components of the **gradient of the loss function** with respect to each component of the object `W`, and with respect to `b`! So as promised, `Flux` has done the hard work of calculating derivatives for us!

*Bonus info*:

To do so, internally it sets up a "computational graph" and propagates the information on derivatives backwards through the graph. Each node of the graph knows which objects feed into that node, so it tells them to also update their gradients, etc.

### Stochastic gradient descent

We can now use these features to reimplement stochastic gradient descent, following the method we used in the previous notebook, but now using backpropagation!

**Exercise:** 

Modify the code from the previous notebook for stochastic gradient descent to use Flux instead.

In [89]:
mypredict(x) = σ.(W*x + b)
loss(x, y) = sum((mypredict(x) .- y).^2)

loss (generic function with 1 method)

In [90]:
x, y = [0.413759, 0.692204], [0.845677]
loss(x, y)

Tracked 0-dimensional Array{Float64,0}:
0.0748059

In [164]:
function stochastic_gradient_descent(loss, W, b, xs, ys, N=1000)

    η = 0.01
    
    for step in 1:N

        i = rand(1:length(xs))  # choose a data point

        x = xs[i]
        y = ys[i]

        l = loss(x, y)
        back!(l)

        b.data -= η * b.grad
        W.data -= η * W.grad
    end
    
    return W, b
    
end

stochastic_gradient_descent (generic function with 2 methods)

In [165]:
function david_stochastic_gradient_descent(loss, W, b, xs, ys, N=1000)

    η = 0.01

    for i in 1:N
        
        which = rand(1:length(xs))  # choose a data point
        
        xx = xs[which]
        yy = ys[which]
        
        l = loss(xx, yy)
        back!(l)
        
        W.data -= η * W.grad
        b.data -= η * b.grad
    
    end
    
    return W, b
    
end
    

david_stochastic_gradient_descent (generic function with 2 methods)

Let's look at the values stored in `b` before we run stochastic gradient descent.

In [166]:
b

Tracked 1-element Array{Float64,1}:
 15756.8

After running `stochastic_gradient_descent`,

In [170]:
W_final, b_final = stochastic_gradient_descent(loss, W, b, xs, ys, 100000)

(param([NaN NaN]), param([NaN]))

we can look at the values of `W_final` and `b_final`, which our machine learned to generate our desired classification.

In [151]:
W_final

Tracked 1×2 Array{Float64,2}:
 382.004  335.077

In [47]:
b_final

Tracked 1-element Array{Float64,1}:
 979.446

**Exercise**: Plot the data and the learned function.

**Solution**:

In [48]:
using Plots; gr()

Plots.GRBackend()

Let's draw the function that the network has learned, together with the data:

In [49]:
heatmap(0:0.01:1, 0:0.01:1, (x,y)->mypredict([x, y]).data[1])

scatter!(first.(x_apples), last.(x_apples), m=:cross)
scatter!(first.(x_bananas), last.(x_bananas))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
<image width="454" height="343" xlink:href="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAcYAAAFXCAYAAAA4WO7PAAAE1UlEQVR4nO3VMRGAAADEsAf/MpDJ
Hax1AEOioFuP+7meAQDbtvPrAAD4E2MEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAI
YwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhj
BIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGME
gDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSA
MEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAw
RgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBG
AAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYA
CGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAI
YwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhj
BIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGME
gDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSA
MEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAw
RgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBG
AAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYA
CGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAI
YwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhj
BIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGME
gDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSA
MEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAw
RgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBG
AAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYACGMEgDBGAAhjBIAwRgAIYwSAMEYA
CGMEgDBGAAhjBIAwRgAIYwSAeAEc+AZMWp4WCgAAAABJRU5ErkJggg==
" transform="translate(53, 22)"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range


**Exercise:** Do this plot every so often as the learning process is proceeding in order to have an animation of the process.

### Automation with Flux.jl

We will need to repeat the above process for a lot of different systems.
Fortunately, Flux.jl provides us with tools to automate this!

Flux allows to create a neuron in a simple way:

In [27]:
model = Dense(2, 1, σ)

Dense(2, 1, NNlib.σ)

The `2` and `1` refer to the number of inputs and outputs, and the neuron is defined using the $\sigma$ function.

In [28]:
typeof(model)

Flux.Dense{NNlib.#σ,TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}

We have made an object of type `Dense`, defined by `Flux`, with the name `model`. This represents a "dense neural network layer" (see later for more on neural network layers).
The parameters that will be modified during the learning process live *inside* the `model` object.

**Exercise**: Investigate which variables are live inside the `model` object and what type they are. How does that compare to the call to create the `Dense` object that we started with?

**Solution**:

Use `model.<TAB>` to check what is inside interactively, or

In [29]:
fieldnames(model)

3-element Array{Symbol,1}:
 :σ
 :W
 :b

In [30]:
model.W

Tracked 1×2 Array{Float64,2}:
 -0.180701  0.935811

In [31]:
model.b

Tracked 1-element Array{Float64,1}:
 0.0

The fact that `model.W` and `model.b` are of size $1 \times 2$ and $1$, respectively, comes from the `(2, 1)` pair in the call to the `Dense` constructor when we created `model`.

`model.W` will be multiplied by a vector `x` of length 2, which it is why it needs to be of size $1 \times 2$.

Again, these are tracked arrays so that Flux can calculate their gradients.

We can use the `model` object just like a function to apply it to data:

In [32]:
model(rand(2))

Tracked 1-element Array{Float64,1}:
 0.626436

**Exercise:** Prove to yourself that you understand what is going on when we call `model`. Create two arrays `W` and `b` with the same elements as `model.W` and `model.b`. Use `W` and `b` to generate the same answer that you get when we call `model([.5, .5])`.

We now need to provide Flux with three pieces of information: 

1. A loss function
2. Some training data
3. An optimization method

### Loss (i.e. cost) functions

Flux has various loss functions built in for example the mean-squared error (`mse`) that we have been using:

In [33]:
loss(x, y) = Flux.mse(model(x), y)

loss (generic function with 1 method)

In [34]:
@edit Flux.mse(model(x), y)

Another common one is the cross entropy, `Flux.crossentropy`.

### Data

The data can take a couple of different forms. 
One form is a single **iterator**, consisting of pairs $(x, y)$ of data and labels.
We can achieve this with `zip`.

**Exercise**: Use `zip` to "zip together" `xs` and `ys`. Then use the `collect` function to check what `zip` actually does.

**Solution**: 

In [35]:
data = zip(xs, ys)

Base.Iterators.Zip2{Array{Array{Float64,1},1},Array{Float64,1}}(Array{Float64,1}[[0.708703, 0.641282], [0.648376, 0.553169], [0.647237, 0.553302], [0.647963, 0.55323], [0.647653, 0.554047], [0.648491, 0.553821], [0.647974, 0.554518], [0.649307, 0.554399], [0.648141, 0.554708], [0.64984, 0.555665]  …  [0.524028, 0.452379], [0.523906, 0.452571], [0.523823, 0.4514], [0.522489, 0.449973], [0.517573, 0.444391], [0.515956, 0.441912], [0.517585, 0.444827], [0.510357, 0.436022], [0.508873, 0.43433], [0.528205, 0.440139]], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [36]:
collect(data)

982-element Array{Tuple{Array{Float64,1},Float64},1}:
 ([0.708703, 0.641282], 0.0)
 ([0.648376, 0.553169], 0.0)
 ([0.647237, 0.553302], 0.0)
 ([0.647963, 0.55323], 0.0) 
 ([0.647653, 0.554047], 0.0)
 ([0.648491, 0.553821], 0.0)
 ([0.647974, 0.554518], 0.0)
 ([0.649307, 0.554399], 0.0)
 ([0.648141, 0.554708], 0.0)
 ([0.64984, 0.555665], 0.0) 
 ([0.648446, 0.555576], 0.0)
 ([0.709808, 0.632473], 0.0)
 ([0.650164, 0.555766], 0.0)
 ⋮                          
 ([0.52913, 0.44031], 1.0)  
 ([0.528731, 0.456548], 1.0)
 ([0.524028, 0.452379], 1.0)
 ([0.523906, 0.452571], 1.0)
 ([0.523823, 0.4514], 1.0)  
 ([0.522489, 0.449973], 1.0)
 ([0.517573, 0.444391], 1.0)
 ([0.515956, 0.441912], 1.0)
 ([0.517585, 0.444827], 1.0)
 ([0.510357, 0.436022], 1.0)
 ([0.508873, 0.43433], 1.0) 
 ([0.528205, 0.440139], 1.0)

### Optimization routine

Now we need to tell Flux what kind of optimization routine to use. It has several built in; stochastic gradient descent is called `SGD`. We must pass it two things: a list of parameters to be modified by the optimization routine, and a step size:

In [37]:
opt = SGD([model.W, model.b], 0.01)
# give a list of the parameters that will be modified

(::#71) (generic function with 1 method)

Inside the optimizer is where the gradient calculations will be carried out; we do not see those details, but if you are curious you can, of course, look at the `Flux.jl` source code!

### Training

We now have all the pieces in place to actually train our model (a single neuron) on the  data. `Flux` provides the function `train!`, which performs a single pass through the data and does a single step of optimization using the partial cost function for each data point:

In [38]:
Flux.train!(loss, data, opt)

We can then just repeat this several times to train the network more and hopefully coax it towards the minimum of the cost function:

In [39]:
for i in 1:100
    Flux.train!(loss, data, opt)
end

Now let's look at the parameters after training:

In [40]:
model.W

Tracked 1×2 Array{Float64,2}:
 -5.9046  -5.21273

In [41]:
model.b

Tracked 1-element Array{Float64,1}:
 6.97496

Instead of writing out a list of parameters to modify, `Flux` provides the function `params`, which extracts all available parameters from a model:

In [42]:
opt = SGD(params(model), 0.01)

(::#71) (generic function with 1 method)

In [43]:
params(model)

2-element Array{Any,1}:
 param([-5.9046 -5.21273])
 param([6.97496])         

## Adding more features

**Exercise**: So far we have just used two features, red and green. 

(i) Add a third feature, blue. Plot the new data.

(ii) Train a neuron with 3 inputs and 1 output on the data.

(iii) Visualize the learned function using `contour3d`.